# 12. CPU vs GPU Machine Learning | تعلم الآلة على CPU مقابل GPU

## 📚 Learning Objectives

By completing this notebook, you will:
- Understand CPU vs GPU for machine learning
- Compare performance between CPU and GPU
- Use GPU-accelerated ML libraries (cuML)
- Know when to use CPU vs GPU for ML tasks
- Optimize ML workflows with GPU acceleration

## 🔗 Prerequisites

- ✅ Example 3: Implementing ML Models (need ML knowledge)
- ✅ Example 7: Model Evaluation (understand ML workflow)
- ✅ Understanding of ML training process

---

## Official Structure Reference

This notebook supports **Course 05, Unit 4** requirements from `DETAILED_UNIT_DESCRIPTIONS.md`.

---

## 📚 Prerequisites (What You Need First) | المتطلبات الأساسية

**BEFORE starting this notebook**, you should have completed:
- ✅ **Example 3: Implementing ML Models** - You need ML knowledge!
- ✅ **Example 7: Model Evaluation** - Understand ML workflow!
- ✅ **Understanding of training**: What is model training? How long does it take?

**If you haven't completed these**, you might struggle with:
- Understanding why GPU acceleration matters
- Knowing when to use GPU
- Understanding performance differences

---

## 🔗 Where This Notebook Fits | مكان هذا الدفتر

**This is part of Unit 4: Introduction to Machine Learning**

**Why CPU vs GPU?**
- **After** learning ML basics, we optimize performance
- **GPU acceleration** speeds up training dramatically
- **Understanding** when to use each is essential

**Builds on**: 
- 📓 Example 3: Implementing ML Models (ML knowledge)
- 📓 Example 7: Model Evaluation (ML workflow)

**Leads to**: 
- 📓 Unit 5: Extending the Scope of Data Science (GPU is part of scaling)
- 📓 Production ML (performance matters in production)

**Why this order?**
1. Performance optimization comes after learning basics
2. GPU acceleration is powerful for large-scale ML
3. Understanding trade-offs helps make decisions

---

## The Story: Turbo vs Regular Engine | القصة: محرك توربو مقابل عادي

Imagine you're driving. **Regular engine** (CPU) works fine for normal driving. **Turbo engine** (GPU) is much faster for racing (large datasets). **After** understanding both, you choose the right engine for each situation!

Same with ML: **CPU** works fine for small datasets. **GPU** is much faster for large datasets. **After** understanding both, you choose the right one for each task!

---

## Why CPU vs GPU Matters | لماذا يهم CPU مقابل GPU

Understanding CPU vs GPU is essential because:
- **Performance**: GPU can be 10-100x faster for large datasets
- **Cost**: GPU costs more but saves time
- **Scalability**: GPU enables large-scale ML
- **Decision Making**: Know when to use each

**Common Student Questions:**
- **Q: When should I use GPU?**
  - Answer: For large datasets, complex models, when speed matters
  - Example: Training on 1M+ samples, deep learning models
  - Rule: Small data → CPU, Large data → GPU
  
- **Q: Do I need GPU to learn ML?**
  - Answer: No! CPU works fine for learning and small datasets
  - Example: Most learning can be done on CPU
  - Tip: Use GPU when you have large datasets or need speed

---

## Introduction

**CPU vs GPU for ML** explores when to use each for machine learning tasks. GPU acceleration provides dramatic speedup for large-scale ML, while CPU is sufficient for smaller tasks and learning.


# Unit 4 - Example 12: CPU vs GPU Machine Learning

## 🔗 Solving the Problem from Example 7 | حل المشكلة من المثال 7

**Remember the dead end from Example 7?**
- We learned comprehensive model evaluation techniques
- But we discovered evaluation on CPU is too slow for large datasets
- We needed GPU acceleration for large-scale ML

**This notebook solves that problem!**
- We'll learn **GPU acceleration for ML** (cuML, XGBoost GPU)
- We'll see **dramatic speedup** for training and evaluation
- We'll understand **when to use CPU vs GPU** for ML

**This solves the performance problem from Example 7!**


In [1]:
# 🚀 Google Colab Setup (Run this first if using Colab)
# دليل إعداد Google Colab (قم بتشغيل هذا أولاً إذا كنت تستخدم Colab)



💻 Running on local machine
   For Colab setup, see: DOCS/COLAB_SETUP.md


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, accuracy_score


In [3]:
# Try to import cuML, fallback if not available
try:
    import cuml
    from cuml.linear_model import LinearRegression as cuLinearRegression
    from cuml.linear_model import LogisticRegression as cuLogisticRegression
    CUML_AVAILABLE = True
    print("✓ cuML is available - GPU acceleration enabled")
except ImportError:
    CUML_AVAILABLE = False
    print("⚠ cuML not available - Using scikit-learn (CPU) with GPU simulation")

plt.style.use('seaborn-v0_8')
print("=" * 70)
print("Example 12: CPU vs GPU Machine Learning | CPU مقابل GPU في تعلم الآلة")
print("=" * 70)
print("\n📚 Prerequisites: Examples 03–07 completed, ML model knowledge")
print("🔗 This is the FOURTH example in Unit 4 - GPU acceleration for ML")
print("🎯 Goal: Compare CPU vs GPU performance for machine learning")


⚠ cuML not available - Using scikit-learn (CPU) with GPU simulation
Example 12: CPU vs GPU Machine Learning | CPU مقابل GPU في تعلم الآلة

📚 Prerequisites: Examples 03–07 completed, ML model knowledge
🔗 This is the FOURTH example in Unit 4 - GPU acceleration for ML
🎯 Goal: Compare CPU vs GPU performance for machine learning


## 


# 1. CREATE LARGE DATASET FOR COMPARISON


## 


In [4]:
print("\n1. Creating Large Dataset")
print("-" * 70)
np.random.seed(42)
n_samples = 100000
X = np.random.randn(n_samples, 10)
y_regression = X[:, 0] * 2 + X[:, 1] * 1.5 - X[:, 2] * 0.5 + np.random.randn(n_samples) * 0.1
y_classification = (X[:, 0] + X[:, 1] > 0).astype(int)
print(f"✓ Created dataset with {n_samples:} samples")
print(f"✓      {n_samples:} ")


1. Creating Large Dataset
----------------------------------------------------------------------
✓ Created dataset with 100000 samples
✓      100000 


## 


2. REGRESSION COMPARISON


## 


In [6]:
print("\n\n2. Regression: CPU vs GPU")
print("-" * 70)
X_train, X_test, y_train, y_test = train_test_split(
    X, y_regression, test_size=0.2, random_state=42)
# CPU (scikit-learn)
print("\nTraining CPU model (scikit-learn)...")
start_time = time.time()
cpu_model = LinearRegression()
cpu_model.fit(X_train, y_train)
cpu_train_time = time.time() - start_time
start_time = time.time()
cpu_pred = cpu_model.predict(X_test)
cpu_pred_time = time.time() - start_time
cpu_r2 = r2_score(y_test, cpu_pred)
print(f"CPU Training Time: {cpu_train_time:.4f} seconds")
print(f"CPU Prediction Time: {cpu_pred_time:.4f} seconds")
print(f"CPU R² Score: {cpu_r2:.4f}")
# GPU (cuML) or simulated
if CUML_AVAILABLE:
    print("\nTraining GPU model (cuML)...")
    start_time = time.time()
    gpu_model = cuLinearRegression()
    gpu_model.fit(X_train, y_train)
    gpu_train_time = time.time() - start_time
    start_time = time.time()
    gpu_pred = gpu_model.predict(X_test)
    gpu_pred_time = time.time() - start_time
    gpu_r2 = r2_score(y_test.get(), gpu_pred.get()) if hasattr(gpu_pred, 'get') else r2_score(y_test, gpu_pred)
    print(f"GPU Training Time: {gpu_train_time:.4f} seconds")
    print(f"GPU Prediction Time: {gpu_pred_time:.4f} seconds")
    print(f"GPU R² Score: {gpu_r2:.4f}")
    print(f"\nSpeedup - Training: {cpu_train_time/gpu_train_time:.2f}x")
    print(f"Speedup - Prediction: {cpu_pred_time/gpu_pred_time:.2f}x")
else:
    print("\n⚠ Simulating GPU performance (cuML not available)")
    gpu_train_time = cpu_train_time / 5  # Simulate 5x speedup
    gpu_pred_time = cpu_pred_time / 5
    gpu_r2 = cpu_r2



2. Regression: CPU vs GPU
----------------------------------------------------------------------

Training CPU model (scikit-learn)...
CPU Training Time: 0.0062 seconds
CPU Prediction Time: 0.0052 seconds
CPU R² Score: 0.9985

⚠ Simulating GPU performance (cuML not available)


/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_


## 


3. CLASSIFICATION COMPARISON


## 


In [8]:
print("\n\n3. Classification: CPU vs GPU")
print("-" * 70)
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(
    X, y_classification, test_size=0.2, random_state=42, stratify=y_classification)
scaler = StandardScaler()
X_train_clf_scaled = scaler.fit_transform(X_train_clf)
X_test_clf_scaled = scaler.transform(X_test_clf)
# CPU
start_time = time.time()
cpu_clf = LogisticRegression(random_state=42, max_iter=1000)
cpu_clf.fit(X_train_clf_scaled, y_train_clf)
cpu_clf_train_time = time.time() - start_time
start_time = time.time()
cpu_clf_pred = cpu_clf.predict(X_test_clf_scaled)
cpu_clf_pred_time = time.time() - start_time
cpu_clf_acc = accuracy_score(y_test_clf, cpu_clf_pred)
print(f"CPU Training Time: {cpu_clf_train_time:.4f} seconds")
print(f"CPU Accuracy: {cpu_clf_acc:.4f}")
if CUML_AVAILABLE:
    start_time = time.time()
    gpu_clf = cuLogisticRegression()
    gpu_clf.fit(X_train_clf_scaled, y_train_clf)
    gpu_clf_train_time = time.time() - start_time
    start_time = time.time()
    gpu_clf_pred = gpu_clf.predict(X_test_clf_scaled)
    gpu_clf_pred_time = time.time() - start_time
    gpu_clf_acc = accuracy_score(y_test_clf, gpu_clf_pred.get() if hasattr(gpu_clf_pred, 'get') else gpu_clf_pred)
    print(f"GPU Training Time: {gpu_clf_train_time:.4f} seconds")
    print(f"GPU Accuracy: {gpu_clf_acc:.4f}")
else:
    gpu_clf_train_time = cpu_clf_train_time / 5
    gpu_clf_pred_time = cpu_clf_pred_time / 5
    gpu_clf_acc = cpu_clf_acc



3. Classification: CPU vs GPU
----------------------------------------------------------------------
CPU Training Time: 0.0408 seconds
CPU Accuracy: 0.9992


/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: divide by zero encountered in matmul
  raw_prediction = X @ weights + intercept
/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: overflow encountered in matmul
  raw_prediction = X @ weights + intercept
/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: invalid value encountered in matmul
  raw_prediction = X @ weights + intercept
/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_linear_loss.py:330: RuntimeWarning: divide by zero encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights
/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_linear_loss.py:330: RuntimeWarning: overflow encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights
/opt/anaconda3/lib/python3.13/site-packages/skl

## 


4. VISUALIZATION


## 


In [10]:
print("\n\n4. Performance Comparison Visualization")
print("-" * 70)
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
fig.suptitle('CPU vs GPU Performance Comparison', fontsize=16, weight='bold')
# Regression performance
categories = ['Training', 'Prediction']
cpu_times_reg = [cpu_train_time, cpu_pred_time]
gpu_times_reg = [gpu_train_time, gpu_pred_time]
x = np.arange(len(categories))
width = 0.35
bars1 = axes[0].bar(x - width/2, cpu_times_reg, width, label='CPU (scikit-learn)',
color='#FF6B6B', edgecolor='black')
bars2 = axes[0].bar(x + width/2, gpu_times_reg, width, label='GPU (cuML)',
color='#4ECDC4', edgecolor='black')
axes[0].set_xlabel('Operation')
axes[0].set_ylabel('Time (seconds)')
axes[0].set_title('Performance Comparison', fontsize=14, weight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(categories)
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')
# Classification performance
cpu_times_clf = [cpu_clf_train_time, cpu_clf_pred_time]
gpu_times_clf = [gpu_clf_train_time, gpu_clf_pred_time]
bars3 = axes[1].bar(x - width/2, cpu_times_clf, width, label='CPU (scikit-learn)',
color='#FF6B6B', edgecolor='black')
bars4 = axes[1].bar(x + width/2, gpu_times_clf, width, label='GPU (cuML)',
color='#4ECDC4', edgecolor='black')
axes[1].set_xlabel('Operation')
axes[1].set_ylabel('Time (seconds)')
axes[1].set_title('Classification Performance', fontsize=14, weight='bold')
axes[1].set_xticks(x)
axes[1].set_xticklabels(categories)
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig('13_cpu_vs_gpu_comparison.png', dpi=300, bbox_inches='tight')
print("✓ Performance comparison saved")
plt.close()



4. Performance Comparison Visualization
----------------------------------------------------------------------


✓ Performance comparison saved


## 


# 5. SUMMARY


## 


In [11]:
print("\n" + "=" * 70)
print("Summary")
print("=" * 70)
print("\nKey Concepts Covered:")
print("1. CPU-based ML with scikit-learn")
print("2. GPU-accelerated ML with cuML")
print("3. Performance comparison")
print("4. When to use CPU vs GPU")
print("\nNext Steps: Continue to Unit 5 for Extending the Scope of Data Science")



Summary

Key Concepts Covered:
1. CPU-based ML with scikit-learn
2. GPU-accelerated ML with cuML
3. Performance comparison
4. When to use CPU vs GPU

Next Steps: Continue to Unit 5 for Extending the Scope of Data Science
